# <h1>Homework 3

# <h1> Problem 1

For a binary mixture with two adjustable parameters $A_{12}$ and $A_{21}$ equilibrium relations are <br/>
$p = x_1\;exp(A_{12}(\frac{A_{21}x_2}{A_{12}x_1 + A_{21}x_2})^2)p_{water}^{sat} + x_2\;exp(A_{21}(\frac{A_{12}x_1}{A_{12}x_1 + A_{21}x_2})^2)p_{1,4\;dioxane}^{sat}$
<br/> The saturation pressure is given as,
<br/> $log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3}$
<br/> We have also been given that,
<br/> $x_1 + x_2 = 1$
<br/> A table of measures for $x_1$ and corresponding $p$ is also given
<br/> We need to solve this problem like a least squares problem

# <h2> Part 1

Least Squares formulation
<br/> $min_{A_{12},A_{21}} \sum (p - p(A_{12},A_{21}))^{2}$
<br/> This means that we need to minimize the sum of squared difference all actual $p$ and calculated $p$ by modifying $As$.

#<h2> Part 2

In order to get a solution as close as possible to the optimum solution, we cannot linearize the given model.
<br/> To solve the problem with its non-linearity intact we solve the problem using gradient descent.

In [64]:
def p_sat(a1,a2,a3):
    return 10**(a1 - ((a2)/(a3+20)))

In [65]:
p_water = p_sat(8.07131,1730.63,233.426)
p_diox = p_sat(7.43155,1554.679,240.337)

In [51]:
import torch as t
from torch.autograd import Variable
import math
import numpy as np

In [52]:
def sat(x1,x2,p_water,p_diox,x):
    return x1*t.exp(x[0]*((x[1]*x2)/(x[0]*x1 + x[1]*x2))**2)*p_water + x2*t.exp(x[1]*((x[0]*x1)/(x[0]*x1 + x[1]*x2))**2)*p_diox

In [53]:
x1 = t.tensor(np.array([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]))
x2 = 1-x1

In [54]:
# = x.detach().numpy()

In [55]:
p = t.tensor([28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5])

In [56]:
def square_loss(act,pred):
    loss = act-pred
    loss = loss**2
    return t.sum(loss)

In [66]:
a = 0.0001
x = Variable(t.tensor([0.6,0.4]),requires_grad = True)
#loss = t.nn.MSELoss()
for i in range(1000):
    p_pred = sat(x1,x2,p_water,p_diox,x)
    loss = square_loss(p,p_pred)
    loss.backward()
    with t.no_grad():
        x -=a*x.grad
        x.grad.zero_()
        print(loss.data.numpy())

829.8784011458104
782.5083185937046
737.9230447334312
695.2186429915533
653.9966780412544
614.0831978805876
575.4127369770492
537.9749115270863
501.7877665328191
466.8835181932683
433.3003322052025
401.0773102200709
370.25119717857837
340.8542223151573
312.9124214765891
286.44467253426564
261.46172841189235
237.96586234851392
215.95047138076598
195.40019916785235
176.29092088245784
158.59041410770354
142.25860948742883
127.24853551134845
113.50702646723212
100.975725507213
89.59210343453944
79.29036794184793
70.00265473075368
61.65991094531325
54.19293687219128
47.53316168076506
41.61349874398814
36.368991323228755
31.737343982480155
27.659363056053664
24.079335935569702
20.945156833163608
18.208529948795942
15.825004409296735
13.753894328242579
11.958237908320935
10.404613292694513
9.062976635527408
7.906462767532911
6.911165561808048
6.0558863961011
5.321902664412048
4.692773420865141
4.154087511419102
3.693251353578302
3.299308573049258
2.9627471930350615
2.675325508997691
2.4299343

In [58]:
print(p_pred)

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       dtype=torch.float64, grad_fn=<AddBackward0>)


# <h2> Problem 2

In this problem we are required to minimize,
<br/> $min_{x_{1},x_{2}} (4 - 2.1x_{1}^2 + \frac{x_{1}^4}{3})x_{1}^2 + x_{1}x_{2} + (-4 + 4x_{2}^2)x_{2}^2$
<br/> where, $x_{1} \in [-3,3]$ and $x_{2} \in [-2,2]$

In [134]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

In [94]:
def cost_fun(x):
    return ((4 - 2.1*(x[0]**2) + (x[0]**4)/3)*(x[0]**2)) + x[0]*x[1] + (-4 + 4*(x[1]**2))*(x[1]**2)


In [145]:
space = [Real(-3,3),Real(-2,2)]

res = gp_minimize(cost_fun,space,acq_func="EI",n_calls=50,n_random_starts=5,random_state=123,noise=0.1**2)

-1.0147683823630866
